# Loading the data into the DB, using Data Models (1/3).

Following the concepts on the different [means](/HowToGuides/start/Ingestion) of ingesting the data, we will build an example using the DataModel method in this notebook.

We will use the [Cookbook dataset](https://docs.google.com/spreadsheets/d/1G1HPG3Dxx5W39OD6b74wMHvWupD7N-DLUbV7tD5owx8/edit?usp=sharing) to have the data persisted onto ApertureDB instance.

We merge the first 3 sheets on the source into a json file such that there will be a list of Dishes objects, and each Dish may have multiple ingredients, along with some oher miscellaneous properties.

## Data Models

A popular way to define the schema in python is using pydantic, and we shall use the same to create the associations of our Cookbook.


In [8]:
from typing import List, Optional
from aperturedb.DataModels import  ImageDataModel, IdentityDataModel

class Ingredient(IdentityDataModel):
    Name: str
    other_names: Optional[str] = ""
    macronutrient: Optional[str] = ""
    micronutrient: Optional[str] = ""
    subgroup: Optional[str] = ""
    category: Optional[str] = ""

class Dish(ImageDataModel):
    contributor: str
    name: str
    location: str
    caption: str
    recipe_url: str
    cuisine: str
    dish_id: int
    ingredients: List[Ingredient]


### Create objects of these classes.

We will provision the data using a json file, prepared from [script](https://github.com/aperture-data/Cookbook/blob/main/scripts/create_nested_json.py)

Example line from the json file:

:::note title="Sample from the dishes.json"
```
{
    "dish_id": 1,
    "url": "https://raw.githubusercontent.com/aperture-data/Cookbook/refs/heads/main/images/001 Large.jpeg",
    "type": "main dish",
    "location": "NJ",
    "cuisine": "Indian",
    "recipe_url": "https://www.tarladalal.com/rajma-chawal-punjabi-rajma-chawal-4951r",
    "contributor": "gautam",
    "caption": "Beans with rice",
    "name": "rajma chawal",
    "ingredients": [
      {
        "Name": "red kidney beans",
        "other_names": "rajma",
        "category": "vegetarian",
        "subgroup": "legume",
        "macronutrient": "protein"
      },
      {
        "Name": "rice",
        "other_names": "chawal",
        "category": "vegetarian",
        "subgroup": "grain",
        "macronutrient": "carbohydrates"
      }
    ]
  }
```
:::

These objects can be passed to a function called generate_add_query which takes care of generating the queries that ApertureDB can execute to persist the objects on the DB.

In [ ]:
from aperturedb.Query import generate_add_query
from aperturedb.CommonLibrary import  execute_query, create_connector
import json

with open("dishes.json") as ins:
    client = create_connector()
    dishes = json.load(ins)
    for dish in dishes:

        dish = Dish(**dish)

        query, blobs, _ = generate_add_query(dish)
        result, response, output_blobs = execute_query(client, query, blobs)

        if result != 0:
            print(response)
            break
